In [35]:
# Import libraries
import os
import requests
import boto3
from botocore.exceptions import NoCredentialsError
import pyarrow.parquet as pq
import pandas as pd
from tqdm import tqdm
import geopandas as gpd
from shapely.geometry import Point
import json

In [14]:
# Fonction de téléchargement des données depuis S3 ou une URL avec barre de progression
def download_data(local_path, url=None, s3_path=None, bucket=None, endpoint=None):
    """
    Télécharge un fichier depuis S3 ou une URL et le sauvegarde localement.
    """
    # Créer le dossier si nécessaire
    dir_path = os.path.dirname(local_path)
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
    
    # Vérifier si le fichier existe déjà
    if os.path.exists(local_path):
        print(f"Les données sont déjà téléchargées : {local_path}")
        return

    # Télécharger depuis S3
    if s3_path and bucket and endpoint:
        print("Téléchargement depuis le bucket S3...")
        try:
            s3 = boto3.client('s3', endpoint_url=endpoint)
            s3.download_file(bucket, s3_path, local_path)
            print(f"Téléchargement effectué avec succès depuis S3 : {local_path}")
        except NoCredentialsError as e:
            print(f"Erreur d'authentification S3 : {e}")
        except Exception as e:
            print(f"Erreur lors du téléchargement depuis S3 : {e}")
            return

    # Télécharger depuis une URL
    elif url:
        print("Téléchargement depuis l'URL...")
        try:
            response = requests.get(url, stream=True)
            response.raise_for_status()
            
            # Barre de progression
            total_size = int(response.headers.get('content-length', 0))
            with open(local_path, 'wb') as f, tqdm(
                desc=f"Téléchargement {os.path.basename(local_path)}",
                total=total_size,
                unit='B',
                unit_scale=True,
                unit_divisor=1024,
            ) as bar:
                for data in response.iter_content(chunk_size=1024):
                    f.write(data)
                    bar.update(len(data))
            print(f"Téléchargement effectué avec succès : {local_path}")
        except requests.RequestException as e:
            print(f"Erreur lors du téléchargement depuis l'URL : {e}")
            return

    else:
        print("Ni URL, ni chemin S3 spécifié. Impossible de télécharger les données.")
        return

# Fonction pour lire les fichiers Parquet
def read_parquet_data(local_path):
    if os.path.exists(local_path):
        try:
            data = pq.read_table(local_path).to_pandas()
            print(f"Données Parquet chargées avec succès depuis : {local_path}")
            return data
        except Exception as e:
            print(f"Erreur lors de la lecture du fichier Parquet : {e}")
    else:
        print(f"Le fichier n'existe pas : {local_path}")
    return None

# Fonction pour lire les fichiers CSV
def read_csv_data(local_path):
    if os.path.exists(local_path):
        try:
            data = pd.read_csv(local_path, sep=';')
            print(f"Données CSV chargées avec succès depuis : {local_path}")
            return data
        except Exception as e:
            print(f"Erreur lors de la lecture du fichier CSV : {e}")
    else:
        print(f"Le fichier n'existe pas : {local_path}")
    return None

In [15]:
# Fonction de téléchargement des données depuis S3 ou une URL avec barre de progression
def download_data(local_path, url=None, s3_path=None, bucket=None, endpoint=None):
    """
    Télécharge un fichier depuis S3 ou une URL et le sauvegarde localement.
    """
    # Créer le dossier si nécessaire
    dir_path = os.path.dirname(local_path)
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
    
    # Vérifier si le fichier existe déjà
    if os.path.exists(local_path):
        print(f"Les données sont déjà téléchargées : {local_path}")
        return

    # Télécharger depuis S3
    if s3_path and bucket and endpoint:
        print("Téléchargement depuis le bucket S3...")
        try:
            s3 = boto3.client('s3', endpoint_url=endpoint)
            s3.download_file(bucket, s3_path, local_path)
            print(f"Téléchargement effectué avec succès depuis S3 : {local_path}")
        except NoCredentialsError as e:
            print(f"Erreur d'authentification S3 : {e}")
        except Exception as e:
            print(f"Erreur lors du téléchargement depuis S3 : {e}")
            return

    # Télécharger depuis une URL
    elif url:
        print("Téléchargement depuis l'URL...")
        try:
            response = requests.get(url, stream=True)
            response.raise_for_status()
            
            # Barre de progression
            total_size = int(response.headers.get('content-length', 0))
            with open(local_path, 'wb') as f, tqdm(
                desc=f"Téléchargement {os.path.basename(local_path)}",
                total=total_size,
                unit='B',
                unit_scale=True,
                unit_divisor=1024,
            ) as bar:
                for data in response.iter_content(chunk_size=1024):
                    f.write(data)
                    bar.update(len(data))
            print(f"Téléchargement effectué avec succès : {local_path}")
        except requests.RequestException as e:
            print(f"Erreur lors du téléchargement depuis l'URL : {e}")
            return

    else:
        print("Ni URL, ni chemin S3 spécifié. Impossible de télécharger les données.")
        return

# Fonction pour lire les fichiers Parquet
def read_parquet_data(local_path):
    if os.path.exists(local_path):
        try:
            data = pq.read_table(local_path).to_pandas()
            print(f"Données Parquet chargées avec succès depuis : {local_path}")
            return data
        except Exception as e:
            print(f"Erreur lors de la lecture du fichier Parquet : {e}")
    else:
        print(f"Le fichier n'existe pas : {local_path}")
    return None

# Fonction pour lire les fichiers CSV
def read_csv_data(local_path):
    if os.path.exists(local_path):
        try:
            data = pd.read_csv(local_path, sep=';')
            print(f"Données CSV chargées avec succès depuis : {local_path}")
            return data
        except Exception as e:
            print(f"Erreur lors de la lecture du fichier CSV : {e}")
    else:
        print(f"Le fichier n'existe pas : {local_path}")
    return None

In [16]:
# Définir le répertoire courant
os.chdir('/home/onyxia/work/Hackathon_Accessibilite')

# Vérification
print("Nouveau répertoire courant :", os.getcwd())

Nouveau répertoire courant : /home/onyxia/work/Hackathon_Accessibilite


In [17]:
# Téléchargement des fichiers en local

# Depuis une URL
download_data(
    local_path="data/acces_libre.csv",
    url="https://www.data.gouv.fr/fr/datasets/accessibilite-des-etablissements-recevant-du-public-erp-pour-les-personnes-en-situation-de-handicap/#/resources/5b0f44f2-e6ea-4a58-874d-6fe364b40342"
)

download_data(
    local_path="data/classement_hotels.csv",
    url="https://minio.lab.sspcloud.fr/inesh/hackathon-tourisme/export%20hotels.csv"
)


download_data(
    local_path="data/classement_residences_camping.csv",
    url="https://minio.lab.sspcloud.fr/inesh/hackathon-tourisme/export%20residences,%20campings,%20PRL,%20Auberges,%20villages%20de%20vacance.csv"
)


Les données sont déjà téléchargées : data/acces_libre.csv
Les données sont déjà téléchargées : data/classement_hotels.csv
Les données sont déjà téléchargées : data/classement_residences_camping.csv


In [32]:
data_acces_libre = pd.read_csv(
    "data/acces_libre.csv",
    sep=",",
    quotechar='"',
    on_bad_lines="skip",
    encoding="ISO-8859-1",
    low_memory=False  # Désactive le mode mémoire réduite
)

In [26]:
data_classement_hotels = pd.read_csv(
    "data/classement_hotels.csv",
    sep=",",
    quotechar='"',  # Gère les champs entourés de guillemets
    on_bad_lines="skip",  # Ignore les lignes problématiques
    encoding="ISO-8859-1"  # Utilise un encodage alternatif
)

In [27]:
data_classement_residences_camping = pd.read_csv(
    "data/classement_residences_camping.csv",
    sep=",",
    quotechar='"',  # Gère les champs entourés de guillemets
    on_bad_lines="skip",  # Ignore les lignes problématiques
    encoding="ISO-8859-1"  # Utilise un encodage alternatif
)

In [33]:
# Afficher un aperçu des données
print(data_acces_libre.head())

                                     id  \
0  e6db2f76-d4d3-4732-b409-a3ad014482c9   
1  c5afca82-aff0-4680-a270-914a2ea4c727   
2  868bb89a-04c4-4bab-9ede-086c38ee9c22   
3  538a3dbd-bbf5-423d-8958-7701c032f02d   
4  c7d781c0-12cf-4d66-aeef-571d8d4efd4f   

                                                name  postal_code  \
0   007 agent i - Agence immobiliÃ¨re Ã  MontmÃ©lian        73800   
1  007 James Bond maison pour tous y compris hand...        24140   
2                   017 Home Concept (Concept Store)        57500   
3                                           0 5 Sens        81500   
4                                          0'mok@bar        75018   

       commune numero                       voie lieu_dit code_insee  siret  \
0  MontmÃ©lian     12           Avenue de savoie      NaN      73171    NaN   
1     Queyssac    136        Impasse des Hameaux      NaN      24345    NaN   
2  Saint-Avold     17  Rue du President Poincare      NaN        NaN    NaN   
3       La

In [ ]:
print(data_classement_hotels.head())
print(data_classement_residences_camping.head())

In [22]:
# Afficher les noms des colonnes
columns = data_acces_libre.columns.tolist()
print(columns)

['id,name,postal_code,commune,numero,voie,lieu_dit,code_insee,siret,activite,contact_url,site_internet,longitude,latitude,transport_station_presence,stationnement_presence,stationnement_pmr,stationnement_ext_presence,stationnement_ext_pmr,cheminement_ext_presence,cheminement_ext_terrain_stable,cheminement_ext_plain_pied,cheminement_ext_ascenseur,cheminement_ext_nombre_marches,cheminement_ext_reperage_marches,cheminement_ext_sens_marches,cheminement_ext_main_courante,cheminement_ext_rampe,cheminement_ext_pente_presence,cheminement_ext_pente_degre_difficulte,cheminement_ext_pente_longueur,cheminement_ext_devers,cheminement_ext_bande_guidage,cheminement_ext_retrecissement,entree_reperage,entree_vitree,entree_vitree_vitrophanie,entree_plain_pied,entree_ascenseur,entree_marches,entree_marches_reperage,entree_marches_main_courante,entree_marches_rampe,entree_marches_sens,entree_dispositif_appel,entree_dispositif_appel_type,entree_balise_sonore,entree_aide_humaine,entree_largeur_mini,entree_p

In [ ]:
columns = data_classement_hotels.columns.tolist()
print(columns)



In [11]:
columns = data_classement_residences_camping.columns.tolist()
print(columns)

columns = data_classement_hotels.columns.tolist()
print(columns)

['Etablissement - Identifiant public', 'Etablissement - Nom commercial', 'Etablissement - Adresse - Code postal', 'Etablissement - Adresse - Département', 'Etablissement - Adresse - Ligne 1', 'Etablissement - Adresse - Ligne 2', 'Etablissement - Adresse - Ligne 3', 'Etablissement - Adresse - Région', 'Etablissement - Adresse - Ville', 'batiment_principal_nb_etage', 'nb_chambre_1personne', 'nb_chambre_2personnes', 'nb_chambre_3personnes', 'nb_chambre_4personnes', 'nb_chambre_5personnes', 'nb_chambre_6personnes', 'nb_chambre_7personnes', 'nb_chambre_8personnes', 'presence_batiment_annexe', "Capacité d'accueil maximum", 'Hôtels - Nombre de chambres', 'Hôtels - Nombre de chambres accessibles', "Nombre d'employés", 'Classement actuel', 'Classement précédent', 'Date de caducité prévisionnelle', "Date de l'arrêté (ou décision) classement actuel de l'établissement", 'Etablissement permanent / saisonnier', 'Etablissement - Marques']


In [49]:
# Convertir data frame en geopanda
geodata_acces_libre = gpd.GeoDataFrame(
    data_acces_libre, geometry=gpd.points_from_xy(data_acces_libre.longitude, data_acces_libre.latitude),
    crs="EPSG:4326"
)


In [ ]:
df.to_json(path_or_buf=None, orient=None, lines=False, force_ascii=True, indent=None)

In [48]:
data_classement_hotels.to_file("/home/onyxia/work/Hackathon_Accessibilite/data/data_classement_hotels.json", driver = "JSON")

AttributeError: 'DataFrame' object has no attribute 'to_file'

In [51]:
# Convertir en JSON
data_classement_hotels.to_file("/home/onyxia/work/Hackathon_Accessibilite/data/geodata_classement_hotels.json", driver = "JSON")


AttributeError: 'DataFrame' object has no attribute 'to_file'

In [45]:
# Sauvegarder sous s3
gdf.to_file("s3://mhillion/diffusion/Hackathon_Accessibilite/data/data_acces_libre.geojson", driver = "GeoJSON")

In [37]:
# Afficher les valeurs de "sanitaire adapté" dans la source data_acces_libre

# Afficher la liste des valeurs et leur fréquence
frequences = data_acces_libre['accueil_audiodescription'].value_counts(dropna=False)

# Convertir en DataFrame pour un affichage plus clair
frequences_df = frequences.reset_index()
frequences_df.columns = ['Valeur', 'Fréquence']

# Afficher le tableau
print(frequences_df)

                                               Valeur  Fréquence
0                                                 NaN     525548
1                  ["avec_\u00e9quipement_permanent"]        217
2                ["avec_\u00e9quipement_occasionnel"]        118
3                                        ["avec_app"]         95
4                            ["sans_\u00e9quipement"]         55
5      ["avec_\u00e9quipement_permanent", "avec_app"]         13
6   ["avec_\u00e9quipement_permanent", "avec_\u00e...         13
7    ["avec_app", "avec_\u00e9quipement_occasionnel"]          9
8   ["avec_\u00e9quipement_permanent", "avec_app",...          7
9   ["avec_\u00e9quipement_permanent", "avec_app",...          6
10  ["avec_\u00e9quipement_permanent", "sans_\u00e...          4
11  ["avec_\u00e9quipement_permanent", "avec_\u00e...          2
12  ["avec_\u00e9quipement_occasionnel", "sans_\u0...          2
13     ["avec_app", "avec_\u00e9quipement_permanent"]          2
14  ["avec_\u00e9quipemen

In [38]:
# Afficher les valeurs de "Hôtels - Nombre de chambres accessibles" dans la source data_classement_hotels

# Afficher la liste des valeurs et leur fréquence
frequences = data_classement_hotels['Hôtels - Nombre de chambres accessibles'].value_counts(dropna=False)

# Convertir en DataFrame pour un affichage plus clair
frequences_df = frequences.reset_index()
frequences_df.columns = ['Valeur', 'Fréquence']

# Afficher le tableau
print(frequences_df)

KeyError: 'Hôtels - Nombre de chambres accessibles'